# Sobrevivientes del Titanic

### Prensenta: 
- Jhair Paris

El 15 de abril de 1912, el Titanic naufragó después de chocar con un iceberg. Debido a la insuficiencia de botes salvavidas, 1502 de los 2224 pasajeros y tripulantes murieron.

Aunque la suerte jugó un papel en la supervivencia de los viajeros, algunos grupos pudieron tener mayores posibilidades de sobrevivir que otros.

Se desea construir un modelo de ML que permita dar respuesta a la pregunta: ¿Qué tipo de persona tuvo más posibilidades de sobrevivir?. Para esto, se cuenta con información como nombre, edad, sexo, clase del tiquete, ciudad de embarque, entre otros.

## Importar librerías
Revisar las librerías/funciones importadas e intentar inferir el propósito de cada una.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import math

## Cargar el conjunto de datos
Dar un vistazo a los datos. ¿De qué características se dispone? ¿Qué particularidades encuentra en los datos? ¿Cuántos ejemplos se tiene?

SibSp: Número de hermanos/esposo(a) abordo.

Parch: Número de padres/hijos abordo

Embarked: Ciudad de embarque.

También, puede descargar el archivo del enlace y explorarlo usando excel.

In [2]:
!curl -L -o titanic.csv "https://www.dropbox.com/s/g19rqwd53co5dh1/titanic.csv?dl=0"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    57    0    57    0     0     10      0 --:--:--  0:00:05 --:--:--    12
100   318  100   318    0     0     54      0  0:00:05  0:00:05 --:--:--     0
100 59541  100 59541    0     0   5250      0  0:00:11  0:00:11 --:--:-- 13225


In [3]:
data = pd.read_csv('titanic.csv', sep=';')
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,712.833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30,C148,C


## Descripción del conjunto de datos
Entender las variables e identificar valores anómalos.

In [4]:
print(data.columns)
data.describe()

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


,PassengerId,Survived,Pclass,Age,SibSp,Parch
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000


## Filtrar y transformar características y etiquetas

#### Identificar y eliminar columnas irrelevantes
¿Cuáles columnas son irrelevantes? Elimínelas.
Se identificaron que las columnas irrelevantes son:
- Name
- Ticket
- Cabin
- PassengerId

In [5]:
print(data['PassengerId'].nunique())
print(data['Ticket'].nunique())
print(data['Name'].nunique())
print(data['Pclass'].nunique())
print(data['SibSp'].nunique())
print(data['Parch'].nunique())
print(data['Fare'].nunique())
print(data['Cabin'].nunique())
#Número de datos faltantes en el campo Cabin
print(len(data[data['Cabin'].isna()]))

#Eliminar columna <nombre_columna>
del data["Name"]
del data["Ticket"]
del data["Cabin"]
del data["PassengerId"]
data

891
681
891
3
7
7
247
147
687


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.25,S
1,1,1,female,38.0,1,0,712.833,C
2,1,3,female,26.0,0,0,7.925,S
3,1,1,female,35.0,1,0,53.1,S
4,0,3,male,35.0,0,0,8.05,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13,S
887,1,1,female,19.0,0,0,30,S
888,0,3,female,NaN,1,2,23.45,S
889,1,1,male,26.0,0,0,30,C


#### Procesar variables restantes
Analice cada una de las transformaciones realizadas a las variables e intente explicar el propósito de cada una. Analice el conjunto de datos resultante.

In [6]:
# Convertir sexo a numérico
data["Sex"].replace({"male": 0, "female": 1}, inplace=True)
# Normalizar Pclass
data["Pclass"] = data["Pclass"] / data["Pclass"].max()
# Normalizar SibSp (Número de hermanos/esposo(a) abordo)
data["SibSp"] = data["SibSp"] / data["SibSp"].max()
# Normalizar Parch (Número de padres/hijos abordo)
data["Parch"] = data["Parch"] / data["Parch"].max()
# Convertir Fare (Costo del tiquete) a números
data["Fare"] = data["Fare"].apply(lambda x: str(x).replace(".", "")).astype(float)
# Normalizar Fare (Costo del tiquete)
data["Fare"] = data["Fare"] / data["Fare"].max()
# One-hot encoding de punto de embarcación (C = Cherbourg, Q = Queenstown, S = Southampton)
data["E_C"] = (data["Embarked"] == "C").replace({True: 1, False: 0})
data["E_Q"] = (data["Embarked"] == "Q").replace({True: 1, False: 0})
data["E_S"] = (data["Embarked"] == "S").replace({True: 1, False: 0})
del data["Embarked"]
# Convertir edad a numérico
data["Age"] = data["Age"].astype(float)
# Imputar datos faltantes de edad con la media
data["Age"] = data["Age"].fillna(data["Age"].mean())
# Normalizar edad
data["Age"] = data["Age"] / data["Age"].max()
data.describe()

/tmp/ipykernel_9570/3883889496.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data["Sex"].replace({"male": 0, "female": 1}, inplace=True)
/tmp/ipykernel_9570/3883889496.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["Sex"].replace({"male": 0, "female": 1}, inplace=True)
/tmp/ipykernel

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,E_C,E_Q,E_S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,0.383838,0.769547,0.352413,0.371239,0.065376,0.063599,0.024918,0.188552,0.086420,0.722783
std,0.486592,0.278690,0.477990,0.162525,0.137843,0.134343,0.080246,0.391372,0.281141,0.447876
min,0.000000,0.333333,0.000000,0.005250,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.666667,0.000000,0.275000,0.000000,0.000000,0.000031,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.371239,0.000000,0.000000,0.000512,0.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,0.437500,0.125000,0.000000,0.015412,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



## Analizar correlación entre variables
Una característica con una correlación moderada o fuerte con las etiquetas con seguridad será relevante en el modelo de clasificación.

Dos características con una correlación fuerte entre sí proveen información redundante.

¿Qué información destacada se puede observar en la matriz de correlación?
Ej:

*   ¿Qué características tienen una correlación moderada o fuerte con la etiqueta?
*   ¿Los datos sugieren alguna relación (aunque sea débil) entre la edad y el poder adquisitivo de una persona?
*   ¿En qué ciudad parece vivir la gente más adinerada?
*   ¿Qué otras variables están correlacionadas de manera moderada o fuerte?
### Repuestas

1. Características con correlación moderada o fuerte con la etiqueta (Survived):

- Sexo (Sex): 0.543351 (correlación moderada positiva)
- Pclass: -0.338481 (correlación moderada negativa)

2. Relación entre la edad y el poder adquisitivo:

La correlación entre la edad y la tarifa pagada (Fare) es muy débil (0.083297), lo que sugiere que no hay una relación significativa entre la edad y el poder adquisitivo de una persona en este conjunto de datos

3. ¿En qué ciudad parece vivir la gente más adinerada?

La correlación más fuerte (aunque sea débil) entre la tarifa pagada y la ciudad de embarque es positiva para (E_C)

4. Variables con correlación moderada o fuerte:
- SibSp y Parch: 0.414838 (correlación moderada positiva)
- E_C y E_S: -0.778359 (correlación fuerte negativa)
- Parch y Sex: 0.245489 (correlación débil positiva)

In [7]:
data.corr()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,E_C,E_Q,E_S
Survived,1.000000,-0.338481,0.543351,-0.069809,-0.035322,0.081629,0.185894,0.168240,0.003650,-0.155660
Pclass,-0.338481,1.000000,-0.131900,-0.331339,0.083081,0.018443,-0.296741,-0.243292,0.221009,0.081720
Sex,0.543351,-0.131900,1.000000,-0.084153,0.114631,0.245489,0.092542,0.082853,0.074115,-0.125722
Age,-0.069809,-0.331339,-0.084153,1.000000,-0.232625,-0.179191,0.083297,0.032024,-0.013855,-0.027121
SibSp,-0.035322,0.083081,0.114631,-0.232625,1.000000,0.414838,-0.027361,-0.059528,-0.026354,0.070941
Parch,0.081629,0.018443,0.245489,-0.179191,0.414838,1.000000,0.046245,-0.011069,-0.081228,0.063036
Fare,0.185894,-0.296741,0.092542,0.083297,-0.027361,0.046245,1.000000,0.294865,-0.078494,-0.206836
E_C,0.168240,-0.243292,0.082853,0.032024,-0.059528,-0.011069,0.294865,1.000000,-0.148258,-0.778359
E_Q,0.003650,0.221009,0.074115,-0.013855,-0.026354,-0.081228,-0.078494,-0.148258,1.000000,-0.496624
E_S,-0.155660,0.081720,-0.125722,-0.027121,0.070941,0.063036,-0.206836,-0.778359,-0.496624,1.000000


## Separar características de etiquetas
Divide en X y y las características y etiquetas. Calcule el número de ejemplos en cada clase.



In [8]:
y = data['Survived']
del data['Survived']
X = data
print(y.shape)
print(X.shape)
print(y.describe())

(891,)
(891, 9)
count    891.000000
mean       0.383838
std        0.486592
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64


## Crear conjunto de entrenamiento y conjunto de prueba
70% entrenamiento, 30% prueba

¿Cuántos ejemplos hay en cada conjunto?

En entrenamiento hay 623

En prueba hay 268

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=0)
print('Entrenamiento: ', y_train.shape)
print('Prueba: ', y_test.shape)

Entrenamiento:  (623,)
Prueba:  (268,)


## Línea base
Para evaluar el desempeño del modelo de clasificación, se define un clasificador que etiqueta todos los ejemplos del conjunto de prueba como de la clase mayoritaria.

In [10]:
y_baseline = pd.Series([0]*len(y_test))
print(y_baseline)

0      0
1      0
2      0
3      0
4      0
      ..
263    0
264    0
265    0
266    0
267    0
Length: 268, dtype: int64


## Evaluar el desempeño de la línea base
Analizar matriz de confusión, exactitud, precisión y tasa de recuperación.

1. Analisis
- El modelo clasifica correctamente todos los ejemplos de la clase 0, pero falla completamente en identificar los ejemplos de la clase 1.
- La alta precisión y tasa de recuperación para la clase 0 contribuyen a una F1-Score alta para esta clase.
- La falta de predicciones correctas para la clase 1 resulta en precisiones, tasas de recuperación y F1-Scores de 0 para esta clase.
- La exactitud del modelo es 63%, pero esta métrica puede ser engañosa debido a la distribución desbalanceada de las clases.
- Los promedios macro y ponderados reflejan el bajo rendimiento del modelo en general, especialmente para la clase 1.


In [11]:
print(confusion_matrix(y_test, y_baseline))
print(classification_report(y_test, y_baseline))

[[168   0]
 [100   0]]
              precision    recall  f1-score   support

           0       0.63      1.00      0.77       168
           1       0.00      0.00      0.00       100

    accuracy                           0.63       268
   macro avg       0.31      0.50      0.39       268
weighted avg       0.39      0.63      0.48       268



/media/jhair/shared/Coding/University-Jobs/python/.linuxPython/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/media/jhair/shared/Coding/University-Jobs/python/.linuxPython/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/media/jhair/shared/Coding/University-Jobs/python/.linuxPython/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa

## Entrenar un modelo de regresión logística
Analice los coeficientes obtenidos (signo y magnitud). Dé una interpretación en el contexto del problema.

1. Analisis

Analisando los coeficiente obtenidos se puede llegar a que las características positivamente asociadas con la Supervivencia son:

  - **Sexo (mujer):** Incrementa significativamente las probabilidades de supervivencia.
  - **Tarifa (Fare):** Mayores tarifas pagadas están asociadas con mayores probabilidades de supervivencia.

Sin embargo las características negativamente asociadas con la Supervivencia son
  - **Clase (Pclass):** Pasajeros en clases más bajas tienen menores probabilidades de sobrevivir.
  - **Edad (Age):** Los pasajeros más viejos tienen menores probabilidades de sobrevivir.

In [12]:
model = LogisticRegression(penalty='l2', n_jobs=-1)
model.fit(X_train, y_train)
w = np.hstack([np.array([model.intercept_[0]]), model.coef_[0]])
print('w_0',model.intercept_[0])
coefs = pd.DataFrame(w[1:], columns=['w'])
coefs['feat_name'] = X.columns
coefs

w_0 1.5528750119781825


,w,feat_name
0,-2.543296,Pclass
1,2.489201,Sex
2,-1.852322,Age
3,-1.470781,SibSp
4,-0.453228,Parch
5,0.840554,Fare
6,0.111076,E_C
7,0.018381,E_Q
8,-0.380582,E_S


## Clasificar los ejemplos de prueba
Clasifique los objetos del conjunto de prueba usando el modelo entrenado y evalúe el desempeño del clasificador. Use un umbral de 0.5.

Analice la matriz de confusión y las medidas de exactitud, precisión y tasa de recuperación.

1. Anailisis

El modelo tiene un desempeño razonablemente bueno, especialmente en predecir quienes no sobrevivieron (clase 0), pero hay margen de mejora en la predicción de los sobrevivientes (clase 1), indicado por valores ligeramente más bajos de precisión y recall para la clase 1.

In [19]:
threshold = 0.5
proba = model.predict_proba(X_test)
# print(proba)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])
y_pred = (proba['p1'] >= threshold)
y_pred.replace({True:1, False:0}, inplace=True)
print('Real y\n', y_test.describe())
print('Pred. y\n', y_pred.describe())
y_test.reset_index(drop=True, inplace=True)
real_vs_pred = pd.DataFrame()
real_vs_pred['real'] = y_test
real_vs_pred['pred'] = y_pred
print(real_vs_pred)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Real y
 count    134.000000
mean       0.373134
std        0.485452
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64
Pred. y
 count    134.000000
mean       0.380597
std        0.487356
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: p1, dtype: float64
     real  pred
0       0     1
1       1     0
2       1     1
3       0     0
4       0     0
..    ...   ...
129     0     0
130     1     0
131     1     0
132     0     0
133     1     1

[134 rows x 2 columns]
[[69 15]
 [14 36]]
              precision    recall  f1-score   support

           0       0.83      0.82      0.83        84
           1       0.71      0.72      0.71        50

    accuracy                           0.78       134
   macro avg       0.77      0.77      0.77       134
weighted avg       0.78      0.78      0.78       134



/tmp/ipykernel_9570/931196180.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_pred.replace({True:1, False:0}, inplace=True)


## Probar distintos umbrales

¿Qué umbral en {0.2, 0.4, 0.5, 0.6, 0.8} da una mayor exactitud?

El umbral que provee mayor exactitud es 0.4

#### Definir conjunto de validación
70%, 15%, 15%
¿Cuántos ejemplos tiene el conjunto de validación y cuántos el conjunto de prueba?



In [14]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=.50,random_state=100)
print('Validación:', y_val.shape)
print('Prueba:', y_test.shape)

Validación: (134,)
Prueba: (134,)


#### Evaluar para diferentes umbrales

In [25]:
proba = model.predict_proba(X_val)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])

for threshold in [0.2, 0.4, 0.5, 0.6, 0.8]:
  print('Umbral:', threshold)
  y_pred = (proba['p1'] >= threshold)
  y_pred.replace({True:1, False:0}, inplace=True)
  print(confusion_matrix(y_val, y_pred))
  print(classification_report(y_val, y_pred))

Umbral: 0.2
[[49 35]
 [ 4 46]]
              precision    recall  f1-score   support

           0       0.92      0.58      0.72        84
           1       0.57      0.92      0.70        50

    accuracy                           0.71       134
   macro avg       0.75      0.75      0.71       134
weighted avg       0.79      0.71      0.71       134

Umbral: 0.4
[[70 14]
 [ 8 42]]
              precision    recall  f1-score   support

           0       0.90      0.83      0.86        84
           1       0.75      0.84      0.79        50

    accuracy                           0.84       134
   macro avg       0.82      0.84      0.83       134
weighted avg       0.84      0.84      0.84       134

Umbral: 0.5
[[71 13]
 [13 37]]
              precision    recall  f1-score   support

           0       0.85      0.85      0.85        84
           1       0.74      0.74      0.74        50

    accuracy                           0.81       134
   macro avg       0.79      0.79  

/tmp/ipykernel_9570/1519856860.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_pred.replace({True:1, False:0}, inplace=True)
/tmp/ipykernel_9570/1519856860.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_pred.replace({True:1, False:0}, inplace=True)
/tmp/ipykernel_9570/1519856860.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option(

## Evaluación
Evalúe el modelo de mayor exactitud en el conjunto de validación sobre el conjunto de prueba.
Analice los resultados. ¿Cómo se comparan con los obtenidos para el conjunto de validación?

Compare los resultados obtenidos por la regresión logística con los de la línea base.

1. ¿Cómo se comparan con los obtenidos para el conjunto de validación con los de la línea base?

Los resultados del conjunto de validación son considerablemente mejores que los de la línea base en términos de precisión, recall, F1-score, y exactitud. Esto indica que el modelo ha mejorado significativamente en su capacidad para clasificar ambas clases correctamente en el conjunto de validación en comparación con la línea base.

In [22]:
proba = model.predict_proba(X_val)
proba = pd.DataFrame(proba, columns = ['p0', 'p1'])
for threshold in [0.4]:
  print('Umbral:', threshold)
  y_pred = (proba['p1'] >= threshold)
  y_pred.replace({True:1, False:0}, inplace=True)
  print(confusion_matrix(y_val, y_pred))
  print(classification_report(y_val, y_pred))

Umbral: 0.4
[[70 14]
 [ 8 42]]
              precision    recall  f1-score   support

           0       0.90      0.83      0.86        84
           1       0.75      0.84      0.79        50

    accuracy                           0.84       134
   macro avg       0.82      0.84      0.83       134
weighted avg       0.84      0.84      0.84       134



/tmp/ipykernel_9570/1599404044.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_pred.replace({True:1, False:0}, inplace=True)
